# Configuration

Once you've created an app in LangGraph, you likely will want to permit configuration at runtime.

For instance, you may want to let the LLM or prompt be selected dynamically, configure a user's `user_id` to enforce row-level security, etc.

In LangGraph, configuration and other ["out-of-band" communication](https://en.wikipedia.org/wiki/Out-of-band) is done via the [RunnableConfig](https://v02.api.js.langchain.com/interfaces/langchain_core_runnables.RunnableConfig.html), which is always the second positional arg when invoking your application.

Below, we walk through an example of letting you configure a user ID and pick which model to use.

## Setup

This guide will use Anthropic's Claude 3 Haiku and OpenAI's GPT-4o model. We will optionally set our API key
for [LangSmith tracing](https://smith.langchain.com/), which will give us
best-in-class observability.


In [1]:
// Deno.env.set("OPENAI_API_KEY", "sk_...");

// Optional, add tracing in LangSmith
// Deno.env.set("LANGCHAIN_API_KEY", "ls__...");
// Deno.env.set("LANGCHAIN_CALLBACKS_BACKGROUND", "true");
Deno.env.set("LANGCHAIN_TRACING_V2", "true");
Deno.env.set("LANGCHAIN_PROJECT", "Configuration: LangGraphJS");


## Define the graph

We will create an exceedingly simple message graph for this example.


In [2]:
import { BaseMessage, HumanMessage } from "@langchain/core/messages";
import { ChatOpenAI } from "@langchain/openai";
import { ChatAnthropic } from "@langchain/anthropic";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableConfig } from "@langchain/core/runnables";
import { END, MemorySaver, START, StateGraph } from "@langchain/langgraph";

interface IState {
  messages: {
    value: (x: BaseMessage[], y: BaseMessage[]) => BaseMessage[];
    default: () => BaseMessage[];
  };
  userInfo: {
    value: (x: string, y: string) => string;
    default: () => string;
  };
}

// This defines the agent state
const graphState: IState = {
  messages: {
    value: (x: BaseMessage[], y: BaseMessage[]) => x.concat(y),
    default: () => [],
  },
  userInfo: {
    value: (x?: string, y?: string) => {
      return y ? y : x ? x : "N/A";
    },
    default: () => "N/A",
  },
};

const promptTemplate = ChatPromptTemplate.fromMessages([
  ["system", "You are a helpful assistant.\n\n## User Info:\n{userInfo}"],
  ["placeholder", "{messages}"],
]);

const callModel = async (
  state: { messages: Array<BaseMessage>; userInfo: string },
  config: RunnableConfig
) => {
  const { messages, userInfo } = state;
  const modelName = config?.configurable?.model;
  const model =
    modelName === "claude"
      ? new ChatAnthropic({ model: "claude-3-haiku-20240307" })
      : new ChatOpenAI({ model: "gpt-4o" });
  const chain = promptTemplate.pipe(model);
  const response = await chain.invoke(
    {
      messages,
      userInfo,
    },
    config
  );
  return { messages: [response] };
};

const workflow = new StateGraph({
  channels: graphState,
});

const fetchUserInformation = async (
  _: { messages: Array<BaseMessage> },
  config: RunnableConfig
) => {
  const userDB = {
    user1: {
      name: "John Doe",
      email: "jod@langchain.ai",
      phone: "+1234567890",
    },
    user2: {
      name: "Jane Doe",
      email: "jad@langchain.ai",
      phone: "+0987654321",
    },
  };
  const userId = config?.configurable?.user;
  if (userId) {
    const user = userDB[userId];
    if (user) {
      return {
        userInfo: `Name: ${user.name}\nEmail: ${user.email}\nPhone: ${user.phone}`,
      };
    }
  }
  return { userInfo: "N/A" };
};

// Define the two nodes we will cycle between
workflow.addNode("fetchUserInfo", fetchUserInformation);
workflow.addNode("agent", callModel);

// Set the entrypoint as `fetchUserInfo`
// so we can always start from there
workflow.addEdge(START, "fetchUserInfo");
workflow.addEdge("fetchUserInfo", "agent");
workflow.addEdge("agent", END);

// Here we only save in-memory
let memory = new MemorySaver();
const graph = workflow.compile({ checkpointer: memory });


[WARN]: You have enabled LangSmith tracing without backgrounding callbacks.
[WARN]: If you are not using a serverless environment where you must wait for tracing calls to finish,
[WARN]: we suggest setting "process.env.LANGCHAIN_CALLBACKS_BACKGROUND=true" to avoid additional latency.
[WARN]: You have enabled LangSmith tracing without backgrounding callbacks.
[WARN]: If you are not using a serverless environment where you must wait for tracing calls to finish,
[WARN]: we suggest setting "process.env.LANGCHAIN_CALLBACKS_BACKGROUND=true" to avoid additional latency.


StateGraph {
  nodes: {
    fetchUserInfo: RunnableLambda {
      lc_serializable: false,
      lc_kwargs: { func: [AsyncFunction: fetchUserInformation] },
      lc_runnable: true,
      name: undefined,
      lc_namespace: [ "langchain_core", "runnables" ],
      func: [AsyncFunction: fetchUserInformation]
    },
    agent: RunnableLambda {
      lc_serializable: false,
      lc_kwargs: { func: [AsyncFunction: callModel] },
      lc_runnable: true,
      name: undefined,
      lc_namespace: [ "langchain_core", "runnables" ],
      func: [AsyncFunction: callModel]
    }
  },
  edges: Set(3) {
    [ "__start__", "fetchUserInfo" ],
    [ "fetchUserInfo", "agent" ],
    [ "agent", "__end__" ]
  },
  branches: {},
  entryPoint: undefined,
  compiled: true,
  supportMultipleEdges: true,
  channels: {
    messages: BinaryOperatorAggregate {
      lc_graph_name: "BinaryOperatorAggregate",
      value: [],
      operator: [Function: value],
      initialValueFactory: [Function: default]
    },

## Call with config


In [3]:
let config = {
  configurable: {
    model: "openai",
    user: "user1",
  },
};
let inputs = { messages: [new HumanMessage("Could you remind me of my email??")] };
for await (const { messages } of await graph.stream(inputs, {
  ...config,
  streamMode: "values",
})) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}


Could you remind me of my email??
-----

Could you remind me of my email??
-----



Sure, John! Your email is jod@langchain.ai. How can I assist you further?
-----



## Change the config

Now let's try the same input with a different user.

In [4]:
let config = {
    configurable: {
      model: "openai",
      user: "user2",
    },
  };
  let inputs = { messages: [new HumanMessage("Could you remind me of my email??")] };
  for await (const { messages } of await graph.stream(inputs, {
    ...config,
    streamMode: "values",
  })) {
    let msg = messages[messages?.length - 1];
    if (msg?.content) {
      console.log(msg.content);
    } else if (msg?.tool_calls?.length > 0) {
      console.log(msg.tool_calls);
    } else {
      console.log(msg);
    }
    console.log("-----\n");
  }
  

Could you remind me of my email??
-----

Could you remind me of my email??
-----

Of course, Jane! Your email is jad@langchain.ai. If you need any further assistance, feel free to ask!
-----



Check out the [LangSmith Trace (link)](https://smith.langchain.com/public/bbd3561f-c0d1-4886-ae18-a6626c6b8670/r/946098b5-84d3-4456-a03c-5dbc8591e76b) for this run to "see what the LLM sees".